In [ ]:
!pip install vectorbt

In [ ]:
import vectorbt as vbt
import yfinance as yf
import pandas as pd

# パラメータ
ns = 5   # 短期移動平均日数
nl = 25  # 長期移動平均日数

# データ取得（トヨタ）
price = vbt.YFData.download('7203.T', start='2021-01-01', end='2022-03-31').get('Close')

if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
    price = price.iloc[:, 0]  # 最初の銘柄の「Close」列だけ使う

# 移動平均の計算
smaS = vbt.MA.run(price, window=ns).ma
smaL = vbt.MA.run(price, window=nl).ma

# シグナル定義：クロスオーバー
entries = smaS > smaL
exits = smaS < smaL

# ポートフォリオ作成（シグナルによるバックテスト）
portfolio = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)

# 成績確認
print(portfolio.stats())
portfolio.plot()


/var/folders/x3/wccbb5_s6s183_4fl9f6ybdc0000gn/T/ipykernel_56707/1033371548.py:10: DeprecationWarning:

Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.



AttributeError: 'Series' object has no attribute 'columns'